In [1]:
from tensorflow.keras.models import load_model
import cv2 as cv
import numpy as np

/home/mahijmomin/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mahijmomin/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mahijmomin/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mahijmomin/.local/lib/python3.7/site-packages/tensorflow/python/framework

In [2]:
face=cv.CascadeClassifier("haarcascade_frontalface_default.xml")

In [3]:
model=load_model("15_points_model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
def preprocessing(images):
    images=images/255.0
    images=images.astype(np.float32)
    return images.reshape(1,96,96,1)

In [5]:
cam=cv.VideoCapture(0)
while True:
    ret,frame=cam.read()
    temp=frame.copy()
    temp1=face.detectMultiScale(temp)
    for (x,y,z,w) in temp1:
        cv.rectangle(temp,(x,y),(x+z,y+w),color=(0,255,255))
    #cv.imshow('frame',temp)
    try:
        new_frame=frame[temp1[0][1]:temp1[0][1]+temp1[0][3],temp1[0][0]:temp1[0][0]+temp1[0][2],:].copy()
        original=new_frame.shape
        new_frame=cv.resize(new_frame,(96,96))
        #cv.imshow('new',new_frame)
        gray_frame=cv.cvtColor(new_frame,cv.COLOR_BGR2GRAY)
        gray_frame=preprocessing(gray_frame)
        output=model.predict(gray_frame)
        for i in range(0,len(output[0]),2):
            cv.circle(img=new_frame,
                      center=(int((output[0][i])*48+48),int((output[0][i+1])*48+48)),
                      radius=2,color=(255,255,255))
        #cv.imshow('output',new_frame)
        frame[temp1[0][1]:temp1[0][1]+temp1[0][3],
              temp1[0][0]:temp1[0][0]+temp1[0][2],:]=cv.resize(new_frame,tuple(original)[:-1])
        cv.imshow('frame1',frame)
    except:
        pass
    if(cv.waitKey(1)&0xff==ord('q')):
        break
cam.release()
cv.destroyAllWindows()